# 4 Extract object data

Using YOLOv8 for object detection to track objects in the videos. This will help identify toys and props used in baby joke interactions.


In [5]:
import os
import math
import sys
import time
import pandas as pd
import numpy as np
import cv2
import torch
from ultralytics import YOLO

import sys

project_root = os.path.join("..")
sys.path.append(project_root)

from src.processors.face_processor import extract_faces_from_video, get_facial_stats
from src.utils.io_utils import getProcessedVideos, saveProcessedVideos

In [6]:
# Add these to your imports
from src.config import PATH_CONFIG
from src.utils.notebook_utils import display_config_info, ensure_dir_exists

# Get paths from config
videos_in = PATH_CONFIG['videos_in']
data_out = PATH_CONFIG['data_out']

# Ensure output directory exists
if ensure_dir_exists(data_out):
    print(f"Created output directory: {data_out}")

# Display configuration information
display_config_info(videos_in, data_out, "Processing Configuration")

processedvideos = getProcessedVideos(data_out)
processedvideos.head()


## Processing Configuration
    
| Configuration | Value | Status |
|---------------|-------|--------|
| Input Videos | `c:\Users\caspar\OneDrive\LegoGPI\babyjokes\LookitLaughter.test` | ✅ exists |
| Output Data | `c:\Users\caspar\OneDrive\LegoGPI\babyjokes\data\1_interim` | ✅ exists |
| Video Count | 54 videos | |

You can change these paths by modifying the `PATH_CONFIG` in `src/config.py` 
or by overriding them in this notebook.


Found existing processedvideos.xlsx with 54 rows.


,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,LaughYesNo,Frames,FPS,...,Faces.file,Speech.when,Speech.file,Diary.file,Diary.when,LastError,annotatedVideo,annotated.when,FrameCount,Keypoints.normed
0,2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,No,NaN,14.232999,...,NaN,2025-04-03 19:56:09,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,2025-04-03 20:16:15,NaN,NaN,NaN,216,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...
1,2UWdXP.joke1.rep3.take1.Peekaboo_h265.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,No,NaN,14.263979,...,NaN,2025-04-03 19:56:10,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,2025-04-03 20:16:17,NaN,NaN,NaN,150,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...
2,2UWdXP.joke2.rep1.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,1,1,Funny,No,NaN,12.275790,...,NaN,2025-04-03 19:56:11,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,2025-04-03 20:16:18,NaN,NaN,NaN,89,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...
3,2UWdXP.joke2.rep2.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,No,NaN,13.920731,...,NaN,2025-04-03 19:56:11,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,2025-04-03 20:16:19,NaN,NaN,NaN,95,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...
4,2UWdXP.joke2.rep3.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,No,NaN,14.010793,...,NaN,2025-04-03 19:56:14,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,2025-04-03 20:16:20,NaN,NaN,NaN,132,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...


In [7]:
# Load YOLOv8 model for object detection
object_model = YOLO('yolov8n.pt')  # Uses the nano model for object detection



In [9]:
# Add Objects columns to processedvideos if they don't exist
from src.processors.object_processor import extract_objects_from_video, match_objects_to_persons, normalize_object_coordinates


if "Objects.file" not in processedvideos.columns:
    processedvideos["Objects.file"] = None
if "Objects.when" not in processedvideos.columns:
    processedvideos["Objects.when"] = None
if "Objects.normed" not in processedvideos.columns:
    processedvideos["Objects.normed"] = None
if "Objects.matched" not in processedvideos.columns:
    processedvideos["Objects.matched"] = None

# Process each video for object detection
force_process = False

for index, row in processedvideos.iterrows():
    #testing only on first 5 videos
    if index > 5:
        break
    if force_process or pd.isnull(row["Objects.file"]):
        try:
            # Get video path
            video_path = os.path.join(videos_in, row["VideoID"])
            
            # Extract objects
            objects_df = extract_objects_from_video(video_path, object_model)
            
            # Save objects data
            stemname = os.path.splitext(row["VideoID"])[0]
            objects_path = os.path.join(data_out, f"{stemname}.objects.csv")
            objects_df.to_csv(objects_path, index=False)
            
            # Update record
            processedvideos.at[index, "Objects.file"] = objects_path
            processedvideos.at[index, "Objects.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
            
            # Normalize coordinates
            if len(objects_df) > 0:
                normed_df = normalize_object_coordinates(objects_df, row["Height"], row["Width"])
                normed_path = os.path.join(data_out, f"{stemname}.objects_normed.csv")
                normed_df.to_csv(normed_path, index=False)
                processedvideos.at[index, "Objects.normed"] = normed_path
            
            # Match objects to persons if keypoints exist
            if not pd.isnull(row["Keypoints.normed"]) and len(objects_df) > 0:
                poses_df = pd.read_csv(row["Keypoints.normed"])
                matched_df = match_objects_to_persons(normed_df, poses_df)
                matched_path = os.path.join(data_out, f"{stemname}.objects_matched.csv")
                matched_df.to_csv(matched_path, index=False)
                processedvideos.at[index, "Objects.matched"] = matched_path
            
            print(f"Processed objects for {row['VideoID']}")
        except Exception as e:
            print(f"Error processing objects for {row['VideoID']}: {e}")
    else:
        print(f"Already processed objects for {row['VideoID']}")

saveProcessedVideos(processedvideos, data_out)
processedvideos[['VideoID', 'Objects.file', 'Objects.normed', 'Objects.matched']].head()

Processing video for objects: ..\LookitLaughter.test\2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Error processing objects for 2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4: 'bytetrack' does not exist
Processing video for objects: ..\LookitLaughter.test\2UWdXP.joke1.rep3.take1.Peekaboo_h265.mp4


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and 

,VideoID,Objects.file,Objects.normed,Objects.matched
0,2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4,None,None,None
1,2UWdXP.joke1.rep3.take1.Peekaboo_h265.mp4,None,None,None
2,2UWdXP.joke2.rep1.take1.NomNomNom_h265.mp4,None,None,None
3,2UWdXP.joke2.rep2.take1.NomNomNom_h265.mp4,None,None,None
4,2UWdXP.joke2.rep3.take1.NomNomNom_h265.mp4,None,None,None
